In [ ]:
!pip install transformers seqeval[gpu]
!pip install torch
!pip install protobuf
!pip install sentencepiece

     |████████████████████████████████| 4.0 MB 25.2 MB/s 
     |████████████████████████████████| 43 kB 2.7 MB/s 
     |████████████████████████████████| 6.6 MB 54.4 MB/s 
     |████████████████████████████████| 596 kB 70.2 MB/s 
     |████████████████████████████████| 77 kB 8.7 MB/s 
     |████████████████████████████████| 895 kB 72.3 MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16180 sha256=ace887d056d9d3d277baff5736209b83e82ebbc01952d236d5e57e8e6cb850d3
  Stored in directory: /root/.cache/pip/wheels/05/96/ee/7cac4e74f3b19e3158dce26a20a1c86b3533c43ec72a549fd7
Successfully built seqeval
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 1.2 MB 28.6 MB/s 


In [ ]:
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
df=pd.read_csv("/content/drive/MyDrive/NER_Final.csv")
df

,Sentence,Tag
0,जियोर्जियोन ०१९। जेपीजी | जियोर्जियोन आंधी ८२...,O O O O O B-CW O O O O O
1,इस क्षेत्र का सबसे प्रसिद्ध व्यंजन बिरिया है ज...,O O O O O O B-PROD O O O O O O O O O O O O O O...
2,राजनीतिक टिप्पणीकार एंड्रयू सुलिवन ने माना कि ...,O O B-PER I-PER O O O O O O O O O O O O
3,लाइटिंग सर्किट से उपकरणों को जोड़ने के लिए एक ...,O O O O O O O O O O O O O O O B-PROD I-PROD O ...
4,तब से उन्हें तीन बार स्थानांतरित किया गया है औ...,O O O O O O O O O O O O O O O B-LOC O O O
...,...,...
15295,चाओ फ्रया नदी मेक्सिको,B-LOC I-LOC I-LOC O
15296,चाय बोर्ड भारत संगठन,B-GRP I-GRP I-GRP O
15297,कंप्यूटर अंकुरण,O B-PROD
15298,एयर इंडिया रीजनल . की समीक्षाएं,B-CORP I-CORP I-CORP O O O


In [ ]:
Tags = list(df.Tag)

In [ ]:
unique_tags=[]
for i in Tags:
  i=i.split(" ")
  for j in i:
    if j not in unique_tags:
      unique_tags.append(j)

In [ ]:
tag_map = {}
id_to_lab = {}
for ind,i in enumerate(unique_tags):
  id_to_lab[ind] = i
  tag_map[i] = ind

In [ ]:
id_to_lab
tag_map

{'B-CORP': 7,
 'B-CW': 1,
 'B-GRP': 9,
 'B-LOC': 6,
 'B-PER': 3,
 'B-PROD': 2,
 'I-CORP': 8,
 'I-CW': 11,
 'I-GRP': 10,
 'I-LOC': 12,
 'I-PER': 4,
 'I-PROD': 5,
 'O': 0}

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoModel, AutoTokenizer, BertForTokenClassification

tokenizer = AutoTokenizer.from_pretrained('ai4bharat/indic-bert')
model = BertForTokenClassification.from_pretrained('ai4bharat/indic-bert',num_labels=len(id_to_lab))

Downloading:   0%|          | 0.00/507 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/5.38M [00:00<?, ?B/s]

You are using a model of type albert to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.


Downloading:   0%|          | 0.00/129M [00:00<?, ?B/s]

Some weights of the model checkpoint at ai4bharat/indic-bert were not used when initializing BertForTokenClassification: ['predictions.dense.weight', 'albert.encoder.albert_layer_groups.0.albert_layers.0.attention.query.bias', 'sop_classifier.classifier.weight', 'albert.encoder.albert_layer_groups.0.albert_layers.0.attention.value.weight', 'albert.encoder.albert_layer_groups.0.albert_layers.0.full_layer_layer_norm.bias', 'albert.encoder.albert_layer_groups.0.albert_layers.0.attention.key.weight', 'albert.encoder.albert_layer_groups.0.albert_layers.0.attention.value.bias', 'albert.embeddings.token_type_embeddings.weight', 'albert.encoder.albert_layer_groups.0.albert_layers.0.attention.query.weight', 'predictions.LayerNorm.weight', 'albert.encoder.albert_layer_groups.0.albert_layers.0.ffn_output.bias', 'albert.encoder.albert_layer_groups.0.albert_layers.0.full_layer_layer_norm.weight', 'albert.embeddings.LayerNorm.bias', 'albert.encoder.albert_layer_groups.0.albert_layers.0.attention.Lay

In [ ]:
train_size = 0.6
train_data = df.sample(frac=train_size,random_state=200)
rem_data = df.drop(train_data.index).reset_index(drop=True)
val_data = rem_data.sample(frac=0.5,random_state=200)
test_data = rem_data.drop(val_data.index).reset_index(drop=True)
train_data.head()

,Sentence,Tag
6441,यह आर्कटिक बंदर कौन हैं? विस्तारित खेल का लीड ...,O B-CW I-CW I-CW I-CW B-CW I-CW O O O O O
5820,हेयरस्टन ने ओहियो उत्तरी विश्वविद्यालय पर कॉले...,O O B-GRP I-GRP I-GRP O O O O
3236,स्टोलर लंबे समय से जर्मेनटाउन हिल्स का निवासी है।,O O O O B-LOC I-LOC O O O
7221,एंडरसन शो जीतने वाले दूसरे ट्रांसजेंडर प्रतियो...,O O O O O O O O O B-PER I-PER O O O O O O
6957,तीसरा था सिंहासन जिसके साथ एक म्यूजिक वीडियो ...,O O B-CW O O O O O O O O O O O O O O O O O O O


In [ ]:
train_data = train_data.reset_index()
train_data.drop(['index'],axis = 1,inplace=True)
train_data.head()
val_data = train_data.reset_index()
val_data.drop(['index'],axis = 1,inplace=True)
val_data.head()
test_data = train_data.reset_index()
test_data.drop(['index'],axis = 1,inplace=True)
test_data.head()

,Sentence,Tag
0,यह आर्कटिक बंदर कौन हैं? विस्तारित खेल का लीड ...,O B-CW I-CW I-CW I-CW B-CW I-CW O O O O O
1,हेयरस्टन ने ओहियो उत्तरी विश्वविद्यालय पर कॉले...,O O B-GRP I-GRP I-GRP O O O O
2,स्टोलर लंबे समय से जर्मेनटाउन हिल्स का निवासी है।,O O O O B-LOC I-LOC O O O
3,एंडरसन शो जीतने वाले दूसरे ट्रांसजेंडर प्रतियो...,O O O O O O O O O B-PER I-PER O O O O O O
4,तीसरा था सिंहासन जिसके साथ एक म्यूजिक वीडियो ...,O O B-CW O O O O O O O O O O O O O O O O O O O


In [ ]:
val_data

,Sentence,Tag
0,यह आर्कटिक बंदर कौन हैं? विस्तारित खेल का लीड ...,O B-CW I-CW I-CW I-CW B-CW I-CW O O O O O
1,हेयरस्टन ने ओहियो उत्तरी विश्वविद्यालय पर कॉले...,O O B-GRP I-GRP I-GRP O O O O
2,स्टोलर लंबे समय से जर्मेनटाउन हिल्स का निवासी है।,O O O O B-LOC I-LOC O O O
3,एंडरसन शो जीतने वाले दूसरे ट्रांसजेंडर प्रतियो...,O O O O O O O O O B-PER I-PER O O O O O O
4,तीसरा था सिंहासन जिसके साथ एक म्यूजिक वीडियो ...,O O B-CW O O O O O O O O O O O O O O O O O O O
...,...,...
9175,बैले . में उपयोग की जाने वाली नृत्य पोशाक,B-CW O O O O O O O O
9176,स्टीमर झीलों और नदियों पर छोटी द्वीपीय भाप से...,B-PROD O O O O O O O O O O O O O O O O O
9177,प्रतिद्वंदी के मावाशी को बाहर और नीचे खींचते स...,O O B-PROD O O O O O O O O O O O O O O O O O O
9178,प्रेम धवन ने इस फिल्म के लिरिक्स और म्यूजिक दो...,B-PER I-PER O O O O O O O O O O O O


In [ ]:
MAX_LEN = 128
TRAIN_BATCH_SIZE = 4
VALID_BATCH_SIZE = 2
EPOCHS = 1
LEARNING_RATE = 1e-05
MAX_GRAD_NORM = 10


In [ ]:
class dataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.len = len(dataframe)
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __getitem__(self, index):
        # step 1: get the sentence and word labels 
        # print("yes")
        sentence = self.data.Sentence[index].strip() 
        word_labels = self.data.Tag[index].split(" ") 

        # step 2: use tokenizer to encode sentence (includes padding/truncation up to max length)
        # BertTokenizerFast provides a handy "return_offsets_mapping" functionality for individual tokens
        encoding = self.tokenizer(sentence,
                             return_offsets_mapping=True, 
                             padding='max_length', 
                             truncation=True, 
                             max_length=self.max_len)

        # step 3: create token labels only for first word pieces of each tokenized word
        labels = [tag_map[label] for label in word_labels] 
        # code based on https://huggingface.co/transformers/custom_datasets.html#tok-ner
        # create an empty array of -100 of length max_length
        encoded_labels = np.ones(len(encoding["offset_mapping"]), dtype=int) * -100

        # set only labels whose first offset position is 0 and the second is not 0
        i = 0
        for idx, mapping in enumerate(encoding["offset_mapping"]):
            if mapping[0] == 0 and mapping[1] != 0:
            # overwrite label
                encoded_labels[idx] = labels[i]
                i += 1

        # step 4: turn everything into PyTorch tensors
        item = {key: torch.as_tensor(val) for key, val in encoding.items()}
        item['labels'] = torch.as_tensor(encoded_labels)

        return item

    def __len__(self):
        return self.len

In [ ]:
training_set=dataset(train_data, tokenizer, MAX_LEN)
valid_set=dataset(val_data, tokenizer, MAX_LEN)
testing_set=dataset(test_data, tokenizer, MAX_LEN)

In [ ]:
training_set[1]

{'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0]),
 'input_ids': tensor([     2,   4384,  10495,    494,  29319,    236,  76930,   1134, 118104,
           9327,  77636,   1976, 109832,  28689,   1134,     37,   2899,   1325,
          39594,  12086,  32721,     15,      3,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,   

In [ ]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

In [ ]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
print(device)

cuda


In [ ]:
model.to(device)

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(200000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_a

In [ ]:
inputs = training_set[2]
input_ids = inputs["input_ids"].unsqueeze(0)
attention_mask = inputs["attention_mask"].unsqueeze(0)
labels = inputs["labels"].unsqueeze(0)

input_ids = input_ids.to(device)
attention_mask = attention_mask.to(device)
labels = labels.to(device)

outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
initial_loss = outputs[0]
initial_loss

tensor(2.2654, device='cuda:0', grad_fn=<NllLossBackward0>)

In [ ]:
optimizer = torch.optim.Adam(params=model.parameters(), lr=LEARNING_RATE)

In [ ]:
# Defining the training function on the 80% of the dataset for tuning the bert model
def train(epoch):
    tr_loss, tr_accuracy = 0, 0
    nb_tr_examples, nb_tr_steps = 0, 0
    tr_preds, tr_labels = [], []
    # put model in training mode
    model.train()
    
    for idx, batch in enumerate(training_loader):
        
        ids = batch['input_ids'].to(device, dtype = torch.long)
        mask = batch['attention_mask'].to(device, dtype = torch.long)
        labels = batch['labels'].to(device, dtype = torch.long)

        output = model(input_ids=ids, attention_mask=mask, labels=labels)
        loss =output["loss"]
        tr_logits = output["logits"]
        tr_loss = loss.item()

        nb_tr_steps += 1
        nb_tr_examples += labels.size(0)
        
        if idx % 100==0:
            loss_step = tr_loss/nb_tr_steps
            print(f"Training loss per 100 training steps: {loss_step}")
           
        # compute training accuracy
        flattened_targets = labels.view(-1) # shape (batch_size * seq_len,)
        active_logits = tr_logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
        flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
        
        # only compute accuracy at active labels
        active_accuracy = labels.view(-1) != -100 # shape (batch_size, seq_len)
        #active_labels = torch.where(active_accuracy, labels.view(-1), torch.tensor(-100).type_as(labels))
        
        labels = torch.masked_select(flattened_targets, active_accuracy)
        predictions = torch.masked_select(flattened_predictions, active_accuracy)
        
        tr_labels.extend(labels)
        tr_preds.extend(predictions)

        tmp_tr_accuracy = accuracy_score(labels.cpu().numpy(), predictions.cpu().numpy())
        tr_accuracy += tmp_tr_accuracy
    
        # gradient clipping
        torch.nn.utils.clip_grad_norm_(
            parameters=model.parameters(), max_norm=MAX_GRAD_NORM
        )
        
        # backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    epoch_loss = tr_loss / nb_tr_steps
    tr_accuracy = tr_accuracy / nb_tr_steps
    print(f"Training loss epoch: {epoch_loss}")
    print(f"Training accuracy epoch: {tr_accuracy}")

In [ ]:
for epoch in range(EPOCHS):
    print(f"Training epoch: {epoch + 1}")
    train(epoch)

Training epoch: 1
Training loss per 100 training steps: 2.514239549636841
Training loss per 100 training steps: 0.011084734803379173
Training loss per 100 training steps: 0.005291730017211307


In [ ]:
def valid(model, testing_loader):
    # put model in evaluation mode
    model.eval()
    
    eval_loss, eval_accuracy = 0, 0
    nb_eval_examples, nb_eval_steps = 0, 0
    eval_preds, eval_labels = [], []
    
    with torch.no_grad():
        for idx, batch in enumerate(testing_loader):
            
            ids = batch['input_ids'].to(device, dtype = torch.long)
            mask = batch['attention_mask'].to(device, dtype = torch.long)
            labels = batch['labels'].to(device, dtype = torch.long)
            
            output = model(input_ids=ids, attention_mask=mask, labels=labels)
            loss =output["loss"]
            eval_logits = output["logits"]
            eval_loss += loss.item()

            nb_eval_steps += 1
            nb_eval_examples += labels.size(0)
        
            if idx % 100==0:
                loss_step = eval_loss/nb_eval_steps
                print(f"Validation loss per 100 evaluation steps: {loss_step}")
              
            # compute evaluation accuracy
            flattened_targets = labels.view(-1) # shape (batch_size * seq_len,)
            active_logits = eval_logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
            flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
            
            # only compute accuracy at active labels
            active_accuracy = labels.view(-1) != -100 # shape (batch_size, seq_len)
        
            labels = torch.masked_select(flattened_targets, active_accuracy)
            predictions = torch.masked_select(flattened_predictions, active_accuracy)
            
            eval_labels.extend(labels)
            eval_preds.extend(predictions)
            
            tmp_eval_accuracy = accuracy_score(labels.cpu().numpy(), predictions.cpu().numpy())
            eval_accuracy += tmp_eval_accuracy

    # labels = [id_to_lab[id.item()] for id in eval_labels]
    # predictions = [id_to_lab[id.item()] for id in eval_preds]
    
    eval_loss = eval_loss / nb_eval_steps
    eval_accuracy = eval_accuracy / nb_eval_steps
    print(f"Validation Loss: {eval_loss}")
    print(f"Validation Accuracy: {eval_accuracy}")

    return eval_labels, eval_preds

In [ ]:
labels, predictions = valid(model, testing_loader)

Validation loss per 100 evaluation steps: 3.0452353954315186
Validation loss per 100 evaluation steps: 0.6400345577466495
Validation loss per 100 evaluation steps: 0.6096696111626126
Validation loss per 100 evaluation steps: 0.6409427521423904
Validation loss per 100 evaluation steps: 0.6404422229960645
Validation loss per 100 evaluation steps: 0.6397117727525353
Validation loss per 100 evaluation steps: 0.6281084534402211
Validation loss per 100 evaluation steps: 0.6253170610625381
Validation loss per 100 evaluation steps: 0.6022254258714566
Validation loss per 100 evaluation steps: 0.5957790489254786
Validation loss per 100 evaluation steps: 0.5949091013103676
Validation loss per 100 evaluation steps: 0.5901190206475326
Validation loss per 100 evaluation steps: 0.5833375122410546
Validation loss per 100 evaluation steps: 0.5887747747393369
Validation loss per 100 evaluation steps: 0.5947343273740461
Validation loss per 100 evaluation steps: 0.592039914006742
Validation loss per 100 e

In [ ]:
labels, predictions = valid(model, training_loader)

Validation loss per 100 evaluation steps: 0.1036045178771019
Validation loss per 100 evaluation steps: 0.48919699261758
Validation loss per 100 evaluation steps: 0.5654408990159927
Validation loss per 100 evaluation steps: 0.5573703308838831
Validation loss per 100 evaluation steps: 0.5495760938880775
Validation loss per 100 evaluation steps: 0.5735067175436364
Validation loss per 100 evaluation steps: 0.5764039617551711
Validation loss per 100 evaluation steps: 0.5841582327102645
Validation loss per 100 evaluation steps: 0.5818067011019371
Validation loss per 100 evaluation steps: 0.579132842421728
Validation loss per 100 evaluation steps: 0.5828718869273758
Validation loss per 100 evaluation steps: 0.5797609282726444
Validation loss per 100 evaluation steps: 0.5823396070851409
Validation loss per 100 evaluation steps: 0.5832801771854522
Validation loss per 100 evaluation steps: 0.5812573613348663
Validation loss per 100 evaluation steps: 0.5807065473558278
Validation loss per 100 eva

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(torch.tensor(labels), torch.tensor(predictions),zero_division = 0))

              precision    recall  f1-score   support

           0       0.83      0.99      0.91      7587
           1       0.75      0.02      0.04       272
           2       0.00      0.00      0.00       272
           3       0.77      0.19      0.31       436
           4       0.00      0.00      0.00        31
           5       0.00      0.00      0.00         6
           6       0.34      0.07      0.11       193
           7       0.59      0.24      0.34       310
           8       0.00      0.00      0.00         8
           9       0.92      0.24      0.39       279
          10       0.33      0.11      0.17         9
          11       0.00      0.00      0.00         5
          12       0.00      0.00      0.00        10

    accuracy                           0.83      9418
   macro avg       0.35      0.14      0.17      9418
weighted avg       0.78      0.83      0.77      9418

